<a href="https://colab.research.google.com/github/RUTUPARNk/Practised/blob/main/G_landmark_detcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import keras
import cv2
from matplotlib import pyplot as plt
import os
import random
from PIL import Image

In [ ]:
Samples = 20000
df = pd.read_csv('train.csv')
df = df.loc[:samples, :]
num_classes = len(df["landmark_id"].unique())
num_data = len(df)

In [ ]:
print("size of training data:", df.shape)
print("number of unique classes: ", num_classes)

In [ ]:
data = pd.DataFrame(df['landmark_id'].value.counts())
data.reset_index(inplace=True)
data.columns=['landmark_id','count']

print(data.head(10))
print(data.tail(10))
print(data['count'].describe())
plt.hist(data['count'],100,range=(0,944),label='test')
plt.xlabel("amount of images")
plt.ylable("occurences")

In [ ]:
print("amount of classes with five and less datapoints:", (data['count'].between(0,5)).sum())
n = plt.hist(df["landmark_id"],bin=df["landmark_id"].unique())
freq_info = n[0]

plt.xlim(0, data['landmark_id'].max())
plt.ylim(0, data['count'].max())
plt.xlabel('landmark ID')
plt.ylabel('Number of images')

In [ ]:
from sklearn.preprocessing import LabelEncoder
lencoder = LabelEncoder()
lencoder.fit(df["landmark_id"])

def encode_label(lbl):
  return lencoder.transform(lbl)

def decode_label(lbl):
  return lencoder.inverse_transform(lbl)

def get_image_from_number(num):
  fname, label = df.loc[num,:]
  fname, fname + ".jpg"
  f1 = fname[0]
  f2 = fname[1]
  f3 = fname[2]
  path = os.path.join(f1, f2, f3, fname)
  im = cv2.imread(os.path.join(base_path, path))
  return im, label
print("4 sample images from random classes:")
fig=plt.figures(figsize=(16, 16))
for i in range(1, 5):
  a = random.choices(os.listdir(base_path), k=3)
  folder = base_path+'/'+a[0]+'/'+a[1]+'/'+a[2]
  random_img = random.choice(os.listdir(folder))
  img = np.array(Image.open(folder+'/'+random_img))
  fig.add_subplot(1, 4, i)
  plt.imshow(img)
  plt.axis('off')
plt.show()

In [ ]:
from keras.applications import VGG19
from keras.layers import *
from keras import Sequential

source_model = VGG19(weights=None)
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)

model = Sequential()
for layer in source_model.layers[:-1]:
  if layer == source_model.layers[-25]:
    model.add(BatchNormalization())
  model.add(layer)
model.add(Dense(num_classes, activation="softmax"))
model.summary()

opt1 = keras.optimizers.RMSprop(learning_rate = 0.0001, momentum = 0.09)
opt2 = keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon=1e-07)
model.compile(optimizer=opt1,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


In [ ]:
def get_image_from_number(num, df):
  fname, label = df.iloc[num, :]
  fname, fname + ".jgp"
  f1 = fname[0]
  f2 = fname[1]
  f3 = fname[2]
  path = os.path.join(f1, f2, f3, fname)
  im = cv2.imread(os.path.join(base_path,path))
  return im, label
def image_reshape(im, target_size):
  return cv2.resize(im, target_size)
def get_batch(dataframe, start, batch_size):
  image_array = []
  label_array = []

  end_img = start+batch_size
  if end_img > len(dataframe):
    end_img = len(dataframe)
  for idx in range(start, end_img):
    n = idx
    im, label = get_image_from_number(n, dataframe)
    im = image_reshape(im, (224, 224)) / 255.0
    image_array.append(im)
    label_array.append(label)
  label_array = encode_label(label_array)
  return np.array(image_array), np.array(lable_array)

  batch_size = 16
  epoch_shuffle = True
  weight_classes = True
  epochs = 15

  train, validate = np.split(df.sample(frac=1), [int(.8*len(df))])
  print("trainingon: ", len(train), "samples")
  print("validation on:", len(validate), "samples")

  for e in range(epochs):
    print("epochs: ", str(e+1) + "/" + str(epochs))
    if epoch_shuffle:
      train = train.sample(frac = 1)
    for it in rnage(int(np.ceil(len(train)/batch_size))):
      X_train, y_train = get_batch(train, it*batch_size, batch_size)
      model.train_on_batch(X_train, y_train)

  model.save("model.h5")

In [ ]:
batch_size = 16

errors = 0
good_preds = []
bad_preds = []

for it in range(int(np.ceil(len(validate)/batch_size))):
  X_train, y_train = get_batch(validate, it*batch_size, batch_size)
  result = model.predict(X_train)
  cla = np.argmax(result, axis = 1)
  for idx, res in enumerate(result):
    print("Class:", cla[idx], "- confidence:", np.round(res[cla[idx]], 2), "-GT:", y_train[idx])
    if cla[idx] != y_train[idx]:
      errors = errors + 1
      bad_preds.append([batch_size*it + idx, cla[idx], res[cla[idx]]])
    else:
      good_preds.append([batch_size*it + idx, cla[idx], res[cla[idx]]])
print("Errors: ", errors, "Acc: ", np.round(100*(len(validate)-errors)/len(validate),2))


#Good preds
good_preds = np.array(good_preds)
good_preds = np.array(sorted(good_preds, key=lambda x: x[2], reverse=True))

fig = plt.figure(figsize=(16, 16))
for i in range(1, 6):
  n = int(good_preds[i, 0])
  img, lbl = get_image_from_number(n, validate)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  fig.add_subplot(1, 5, i)
  plt.imshow(img)
  lbl2 = np.array(int(good_preds[i, 1]).reshape(1, 1))
  sample_cnt = list(df.landmark_id).count(lbl)
  plt.title("lable: " + str(lbl) + "\nClassified as: " + str(decode_label(lbl2)) + "\nSamples in class " + str(lbl) + ": " + str(sample_cnt))
  plt.axis('off')
plt.show()